In [1]:
from google.colab import drive
import os, sys
from datetime import datetime
import numpy as np
from tqdm.notebook import tqdm 
import tensorflow as tf
from pytz import timezone
from glob import glob 
import matplotlib.pyplot as plt
import shutil
import cv2


drive.mount('/content/drive')
link_name = '/packages'
target = '/content/drive/My Drive/Colab Notebooks/collage/packages'

try:
  os.symlink(target, link_name)
except:
  os.chdir(target)
  os.remove(link_name)
  os.symlink(target, link_name)
sys.path.insert(0, link_name)

from pororo import Pororo
os.chdir('/content/drive/MyDrive/Colab Notebooks/collage')
import scrap
import cairosvg 
import segsave
# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36
# Mozilla/5.0 (X11; CrOS x86_64 10066.0.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
input user-agent 
Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36


In [2]:
# !pip --version
# !pip install --target=$link_name --upgrade pip
# !pip install --target=$link_name --use-deprecated=legacy-resolver git+https://github.com/kakaobrain/pororo.git
# !pip install --target=$link_name konlpy
# !pip install --target=$link_name python-mecab-ko
# !pip install --target=$link_name kollocate
# !pip install --target=$link_name koparadigm
# !pip install --target=$link_name g2pk
# !pip install --target=$link_name selenium
# !pip install --target=$link_name webdriver_manager
# !pip install --target=$link_name git+https://github.com/naver/kor2vec.git
# !pip install  --target=$link_name cairosvg
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (99.8 kB/s)
Reading package l

In [3]:
from google.colab import output
output.enable_custom_widget_manager()

In [4]:
def return_trend_dict():
  return {'keyword': None, 'namuKeyword': None, 'namuKeyword_kind': None, 'articles': None, 'article_summary':None, 'img_name':None, 'img':None, 'links':None, 'title_picked':None}

In [5]:
now = datetime.now(timezone('Asia/Seoul'))
datetime_text = str(now.strftime('%y-%m-%d-%H:%M'))

keyword_dicts = []
keyword_now = scrap.scrapSignalKeyword()
# keyword_now = ['이상한 변호사 우영우', '하락 서울 6개월만에', '문지효', '담대한 어리석음 상대', '수사한 이원석 총장에', '이상민', '박시은', '트로트', '송파구청장', 'MBC 뉴스데스크']
# keyword_now = ['이상한 변호사 우영우', '강제 북송 검찰', '박시은', '정경심 허리 디스크', '메이플', '정창욱 징역 1년', '이상민', '나는솔로', '이준석', '히든싱어 7 송은이']

print(f'{datetime_text} : {keyword_now}')
for idx, keyword in tqdm(enumerate(keyword_now)):
  dict_ = return_trend_dict()
  articles, links = scrap.scrapNaverNewsKeyword(keyword, article_num=10, sort='1')
  dict_['keyword'] = keyword
  dict_['articles'] = articles
  dict_['img_name'] = str(idx) + '.png'
  dict_['links'] = links
  keyword_dicts.append(dict_)

22-08-20-13:03 : ['코로나19 12만', '빅마우스', '김호중', '진태현', '황희찬', '이재명', '박시은', '美 2022MTV VMA', '일라이', '폭염주의보 경북 호우주의보']


0it [00:00, ?it/s]

In [6]:
def person_replace(txt):
  per_dict = { '尹':'윤석열', '文':'문'}
  for word, replacement in per_dict.items():
    txt = txt.replace(word, replacement)
  return txt

In [7]:
ner = Pororo(task="ner", lang="ko")
se = Pororo(task="sentence_embedding")
summ = Pororo(task="summarization", model="abstractive", lang="ko")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [8]:
def find_namukey(keyword, d=False):
  ner_result = ner(person_replace(keyword))
  ner_person_list = [re[0] for re in ner_result if re[1]=='PERSON']
  ner_or_list = [re[0] for re in ner_result if re[1]=='ORGANIZATION']
  ner_arti_list = [re[0] for re in ner_result if re[1]=='ARTIFACT']
  ner_city_list = [re[0] for re in ner_result if re[1]=='CITY']

  if d:
    if len(ner_person_list) > 0:
      for p in ner_person_list:
        if keyword == p:
          return p, 'person'
      return ner_person_list[0], 'person'

    elif len(ner_or_list) > 0:
      for o in ner_or_list:
        if keyword == o:
          return o, 'organization'
      return ner_or_list[0], 'organization'

    elif len(ner_arti_list) > 0:
      for a in ner_arti_list:
        if keyword == a:
          return a, 'artifact'
      return ner_arti_list[0], 'artifact'

    elif len(ner_city_list) > 0:
      for c in ner_city_list:
        if keyword == c:
          return c, 'city'
      return ner_city_list[0], 'city'
    else :
      return '-', '-'
  else:
    if len(ner_person_list) > 0:
      for p in ner_person_list:
        if keyword == p:
          return p, 'person'
      return ner_person_list[0], 'person'
    else :
      return '-', '-'

In [9]:
for dict_ in tqdm(keyword_dicts):
  dict_['namuKeyword'], dict_['namuKeyword_kind'] = find_namukey(dict_['keyword'])

  0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
print([k['namuKeyword'] for k in keyword_dicts])

['-', '-', '김호중', '진태현', '황희찬', '이재명', '박시은', '-', '일라이', '-']


In [11]:
from sklearn.metrics.pairwise import cosine_similarity

def find_impor_title_idx(dict_):
  titles = [article[0].strip() for article in dict_['articles']]
  titles_embedding = [se(t).reshape(1,-1) for t in titles]

  sims = []
  for i in range(len(titles_embedding)):
    cands = titles_embedding.copy()
    query = cands.pop(i)
    temp_num = 0
    for c in cands:
      temp_num += cosine_similarity(query, c)[0][0]
    sims.append(temp_num)
  max_ = max(sims)
  max_idx = sims.index(max_)

  return max_idx

In [12]:
for dict_ in tqdm(keyword_dicts):
  max_idx = find_impor_title_idx(dict_)
  dict_['title_picked'] = max_idx
  if dict_['namuKeyword']=='-':
    target_title = dict_['articles'][max_idx][0]
    dict_['namuKeyword'], dict_['namuKeyword_kind'] = find_namukey(target_title, d=True)
  dict_['article_summary'] = summ(dict_['articles'][dict_['title_picked']][1])
  

  0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
print([k['namuKeyword'] for k in keyword_dicts])

['-', '임윤아', '김호중', '진태현', '황희찬', '이재명', '박시은', '방탄소년단', '일라이', '서울']


In [14]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/collage/result')

for i,dict_ in enumerate(keyword_dicts):
    print(f"{dict_['keyword']} -> {dict_['namuKeyword']}")
    try:
        os.mkdir('./'+datetime_text)
    except:
        pass
    try:
        os.mkdir(f'./{datetime_text}/img')
        os.mkdir(f'./{datetime_text}/txt')
    except:
        pass
    try:
        os.mkdir(f"./{datetime_text}/txt/{dict_['keyword']}")
    except:
        pass
    txt_path = f"./{datetime_text}/txt/{dict_['keyword']}"
    # scrap.saveArticles(dict_['articles'], txt_path)
    img_path = f'./{datetime_text}/img/{i}.png'
    scrap.scrapNamuImg(dict_['namuKeyword'],img_path, namuKeyword_kind=dict_['namuKeyword_kind'])

코로나19 12만 -> -
빅마우스 -> 임윤아
김호중 -> 김호중
진태현 -> 진태현
황희찬 -> 황희찬
이재명 -> 이재명
박시은 -> 박시은
美 2022MTV VMA -> 방탄소년단
일라이 -> 일라이
폭염주의보 경북 호우주의보 -> 서울


In [15]:
im_list = glob(f'./{datetime_text}/img/*')
try:
    os.mkdir('./'+datetime_text+'/segmentation')
except:
    print('-')
model_present_path = '/content/drive/MyDrive/Colab Notebooks/collage/segmentation/model_weight/base_weight.h5'
model_present = tf.keras.models.load_model(model_present_path)

In [16]:
for dict_ in tqdm(keyword_dicts):
  load_path = f'./{datetime_text}/img/{dict_["img_name"]}'
  save_path = f'./{datetime_text}/segmentation/{load_path.split("/")[-1]}'
  if dict_['namuKeyword_kind'] == 'person':
    try:
      temp_img = segsave.segSave(model_present, load_path, save_path=None)
      temp_img = np.stack([temp_img[:,:,2], temp_img[:,:,1], temp_img[:,:,0], temp_img[:,:,3]], axis=2)
      dict_['img'] = segsave.makeWhiteEdge(temp_img, save_path=save_path, iterations=5)
    except:
      pass
  elif dict_['namuKeyword_kind'] == '-':
    continue   
  else:
    try:
      temp_img = cv2.imread(load_path, cv2.IMREAD_UNCHANGED)
      temp_img = np.stack([temp_img[:,:,2], temp_img[:,:,1], temp_img[:,:,0], temp_img[:,:,3]], axis=2)
      try:
        dict_['img'] = segsave.makeWhiteEdge(temp_img, save_path=save_path, iterations=5)
      except:
        shutil.copy(load_path, save_path)
        dict_['img'] = temp_img
    except:
      pass

  0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
keyword_dicts[-1]

{'keyword': '폭염주의보 경북 호우주의보',
 'namuKeyword': '서울',
 'namuKeyword_kind': 'city',
 'articles': [('[날씨] 서울 폭염주의보, 중부 무더위…남부 오후까지 비',
   '[앵커]서울을 비롯한 일부 수도권과 충청도에 폭염주의보가 내려졌습니다. 주말인 오늘 중부는 무더위가 이어지겠고, 남부지방은 오후까지 비가 내릴텐데요. 자세한 날씨는 기상캐스터 연결해 알아보겠습니다. 구본아 캐스터.[캐스터]이번 주 흐린 날들이 많은데요.아쉬움을 달래 주려는 듯 주말인 오늘은 서울 하늘 보시다시피 이렇게 맑게 갰습니다.오히려 볕이 강하다 못해 뜨거워서 덥다는 느낌이 강한데 오늘 중부지방은 가끔씩 구름만 끼어 있겠고요.낮 더위가 이어지겠습니다.오늘 한낮기온 서울은 32도까지 올라서 어제보다 4도 이상 높겠고요.또 그 밖의 지역 춘천과 청주 32도, 남부는 흐린 하늘 이어지면서 대구와 광주가 30도에 머물겠습니다.한편 남부지방으로는 여전히 비가 내리고 있습니다.일부 전남과 경북 내륙에 호우주의보가 발효 중인데요.오후까지 남부 곳곳에 시간당 30~50mm에 달하는 집중호우가 쏟아지겠고요.앞으로 전남에 많게는 80m 이상, 영남에는 20~70mm 비가 더 내리다가 밤에는 모두 그치겠습니다. 일요일인 내일은 대체로 맑겠고, 서울과 광주 32도까지 오르는 등 전국이 후텁지근하겠습니다. 다음 주 중반부터는 낮 기온이 30도 아래로 떨어지면서, 더위가 한풀 꺾일 것으로 전망하고 있고요. 다음 주 목요일과 금요일 사이에는 충청 이남 지역 중심으로 또 한 번 비 예보 나와 있습니다.알찬 주말 보내시기 바랍니다.지금까지 광화문광장에서 날씨 전해 드렸습니다.(구본아 기상캐스터)#폭염주의보 #호우주의보 #집중호우 연합뉴스TV 기사문의 및 제보 : 카톡/라인 jebo23'),
  ('[날씨] 전남 중심으로 호우주의보…중부 곳곳 폭염특보',
   '오늘(20일) 중부지방 곳곳은 다시 폭염특보가 발효돼 무덥겠고, 남부지방에는 오후까지 강

In [18]:
temp_img.shape

AttributeError: ignored

In [ ]:
dict_['img']

In [ ]:
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
hh = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

import requests
import re
import os 
import numpy as np
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image

header={"user-agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

# def scrapNamuImg(key, path, namuKeyword_kind='person'):
#     driver = scrap.set_chrome_driver()

#     url = 'https://namu.wiki/w/'+key
#     driver.get(url=url)
#     html = driver.page_source
#     driver.close()
#     soup = BeautifulSoup(html, 'lxml')

#     if namuKeyword_kind == 'person':
#       imglink = [d['src'] for d in soup.find_all('img', attrs={'class':'dVTtICxy'}) if not d.find_parent('dd')]
#       for i, link in enumerate(imglink):
#           res=requests.get("https:"+link,headers=header)
#           # print(i)
#           if 'svg' not in res.text and 'video' not in res.text:
#               urlopen_img = Image.open(BytesIO(res.content))
#               counts = np.unique(np.array(urlopen_img.split()[-1]), return_counts=True)[1]
#               ratio = counts[0]/np.sum(counts)
#               if urlopen_img.size[1]*urlopen_img.size[0] > 100000 and ratio<0.3 :
#                   print(res.url)
#                   urlopen_img.save(path,'png')
#                   break
#     else :
#       imglink = [d['src'] for d in soup.find_all('img', attrs={'class':'dVTtICxy'}) if not d.find_parent('dd') and ('로고' in  d['alt'] or'logo' in  d['alt'] or 'CI' in  d['alt'] or 'ci' in  d['alt'])]
#       for i, link in enumerate(imglink):
#           res=requests.get("https:"+link,headers=header)
#           # print(i)
#           if 'svg' in res.text:
#               print(res.url)
#               urlopen_img = Image.open(BytesIO(cairosvg.svg2png(res.content)))
#               urlopen_img.save(path,'png')
#               break
#           else:
#               print(res.url)
#               urlopen_img = Image.open(BytesIO(res.content))
#               urlopen_img.save(path,'png')
#               break

In [ ]:
namuKeyword_kind = 'person'
key = '카세미루'
path = 'asdasd'


driver = scrap.set_chrome_driver()

url = 'https://namu.wiki/w/'+key
driver.get(url=url)
html = driver.page_source
driver.close()
soup = BeautifulSoup(html, 'lxml')

if namuKeyword_kind == 'person':
  imglink = [d['src'] for d in soup.find_all('img', attrs={'class':'dVTtICxy'}) if not d.find_parent('dd')]
  for i, link in enumerate(imglink):
      res=requests.get("https:"+link,headers=header)
      if 'svg' not in res.text and 'video' not in res.text:
          urlopen_img = Image.open(BytesIO(res.content))
          counts = np.unique(np.array(urlopen_img.split()[-1]), return_counts=True)[1]
          ratio = counts[0]/np.sum(counts)
          if urlopen_img.size[1]*urlopen_img.size[0] > 100000 and ratio<0.3 :
              print(res.url)
              urlopen_img.save(path,'png')
              break
else :
  imglink = [d['src'] for d in soup.find_all('img', attrs={'class':'dVTtICxy'}) 
                    if not d.find_parent('dd') and (key in  d['alt']) and
                      ('로고' in  d['alt'] or'logo' in  d['alt']or 'CI' in  d['alt'] or 'ci' in  d['alt'] or '휘장' in d['alt']) ]
  if len(imglink) < 1:
    imglink = [d['src'] for d in soup.find_all('img', attrs={'class':'dVTtICxy'}) 
                      if not d.find_parent('dd') and ('로고' in  d['alt'] or'logo' in  d['alt']or 'CI' in  d['alt'] or 'ci' in  d['alt'] or '휘장' in d['alt']) ]
  for i, link in enumerate(imglink):
      res=requests.get("https:"+link,headers=header)
      # print(i)
      try:
          print(res.url)
          urlopen_img = Image.open(BytesIO(cairosvg.svg2png(res.content)))
          urlopen_img.save(path,'png')
          break
      except:
          print(res.url)
          urlopen_img = Image.open(BytesIO(res.content))
          urlopen_img.save(path,'png')
          break


In [ ]:
html

In [ ]:
soup.find_all('img', attrs={'class':'dVTtICxy'})

In [ ]:
[d['src'] for d in soup.find_all('img', attrs={'class':'dVTtICxy'})]

In [ ]:
re = 'https://w.namu.la/s/c9c48bc9a0e156492e1c9d388ac58dbcec8710ac8cc3d10da9cbd0254018cfc26d66860c68e72a1c439604c4e28ea7f2fa00026ec52bd2327cd6f903a64f477ff2ce84521e0bde7d58528ecee15268a5a6906c8403edbf977d586e5df990438b'
res=requests.get(re,headers=header)

In [ ]:
res=requests.get(re,headers=header)

In [ ]:
img = Image.open(BytesIO(cairosvg.svg2png(res.content)))

In [ ]:
import cv2
img_arr = np.array(img)
im_alpha = img_arr[:,:,3]
kernel = np.ones((3,3), np.uint8)
result = cv2.dilate(im_alpha, kernel, iterations=3)

white_space_mask = np.stack([np.where(result - im_alpha>0, True,False)]*3, axis=2)

img_arr[:,:,3] = result
img_arr[:,:,:3][white_space_mask] = 255

In [ ]:
Image.fromarray(img_arr).save('/content/drive/MyDrive/Colab Notebooks/collage/result/9__.png','png')